## Pseudo Labeling by Predicted ROC Cutoff

Pseudo labeling in this post is basically the same as suggested below reference links,

1. Predict target in test with built model and training data.
2. Add confident predicted test observations to training data. 
3. Predict target with new built model and the training data added confident predicted test observations.

but different method is used to determine confident predicted test observation. 

In this [post](https://www.kaggle.com/cdeotte/pseudo-labeling-qda-0-969), pseudo labeled test observations are determined by predicted probability, $Pr(y=0|x) > 0.99$ and $Pr(y=1|x) > 0.99$. And, in this [post](https://www.kaggle.com/c/siim-isic-melanoma-classification/discussion/168383), values are labeled by intersection from ensemble of predicted values with top and bottom N, since the metric for this competition is AUC score, which is based on rank of the target. 



This kernel will demonstrate pseudo labeling by predicted optimal ROC thresholds.
This is from an idea that the optimal ROC cutoff must be somewhere around the average of predicted target value, then the optimal ROC cutoff for test can be predicted with the optimal ROC cutoff for train and the fitted values for train and test. 
The process is as follows,

1. Predict target in train and test with built model and training data.  
2. Find the optimal ROC cutoff using predicted target in train and real target values for each models. 
3. Predict ROC cutoff for the predicted test target with optimal ROC cutoff and predicted target in train for each models. 
4. Determine confident predicted test observations. 


In this post, the predicted target values are from total 14 models, which are EfficientNets, and the models and data are borrowed from this [post by Chris](https://www.kaggle.com/cdeotte/triple-stratified-kfold-with-tfrecords), but different augmentations and image size. Their CV and Public LB scores are vary around 0.90~0.92

It was just experimental attempt, nevertheless I share this because I found some pattern in doing this and eventually hope this is helpful.

## Reference

AUC score Reference link:
[Mann–Whitney U test](https://en.wikipedia.org/wiki/Mann%E2%80%93Whitney_U_test)

Models + others(Triple Stratified Folds, etc) Reference links:

[Triple Stratified KFold with TFRecords by Chris Deotte](https://www.kaggle.com/cdeotte/triple-stratified-kfold-with-tfrecords)

Pseudo Labeling Reference links: 

[Pseudo Labeling by Chris Deotte](https://www.kaggle.com/cdeotte/pseudo-labeling-qda-0-969)

[Pseudo Labeling by Saksham Gupta](https://www.kaggle.com/c/siim-isic-melanoma-classification/discussion/168383)

In [ ]:
import pandas as pd
import numpy as np
import os
import time
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, roc_curve
from scipy import stats

import matplotlib.pyplot as plt
import seaborn as sns

These are predicted by efficientnets with different settings, such as crops and image size, and the models are borrowed from this [post](https://www.kaggle.com/cdeotte/triple-stratified-kfold-with-tfrecords)

In [ ]:
train = pd.read_csv('../input/siim-isic-melanoma-classification/train.csv')
test = pd.read_csv('../input/siim-isic-melanoma-classification/test.csv')
dpl = pd.read_csv('../input/siim-duplicates/siim_dpl2020.csv') 
#duplicate IDs including 9 more images as Chris mentioned here, https://www.kaggle.com/c/siim-isic-melanoma-classification/discussion/161943

train['is_train'] = 0
test['is_train'] = 1

#removing duplicates in train
train = train[~train['image_name'].isin(dpl['ISIC_id_paired'])]

train = train[['image_name', 'is_train', 'target']]
test = test[['image_name', 'is_train']]

path = '../input/models'

#predicted values for train and test, total 14 models
files_tr = [x for x in os.listdir(path) if x.endswith('_tr.csv')]
files_te = [x for x in os.listdir(path) if x.endswith('_te.csv')]

tr_models = {}
te_models = {}

for i in files_tr:
    tr_models[i] = pd.read_csv(path + '/' + i)
    tr_models[i]['predicted_' + i.replace('_tr.csv', '')] = tr_models[i]['pred']
    tr_models[i] = tr_models[i][['image_name', 'predicted_' + i.replace('_tr.csv', '')]]
    train = train.merge(tr_models[i], on = 'image_name', how = 'left')

for i in files_te:
    te_models[i] = pd.read_csv(path + '/' + i)
    te_models[i]['predicted_' + i.replace('_te.csv', '')] = te_models[i]['target']
    te_models[i] = te_models[i][['image_name','predicted_' + i.replace('_te.csv', '')]]
    
    test = test.merge(te_models[i], on = 'image_name', how = 'left')
    
test['target'] = 0

combine = pd.concat([train,test])

print(train.shape, test.shape)

train.columns

In [ ]:
train.iloc[:,:5].head(5)

In [ ]:
test.iloc[:, :5].head(5)

In [ ]:
predicted_val = ['predicted_model' + str(x) for x in range(1,15)]

## Different distribution, Train vs Test

Predicted target values in train and test have different distributions, and some of you might also notice hard overfitting in models with meta data. Kolmogorov-Smirnov test and T test are used.

##### Since their distributions are different, the ROC cutoff for train and predicted ROC cutoff for test would have similar gap such the differences of their distributions.

In [ ]:
def ttest(col, *, data=combine, group="site", group_val=["1", "2"]):
    res = stats.ttest_ind(
        combine[combine[group] == group_val[0]][col],
        combine[combine[group] == group_val[1]][col],
        equal_var=False,
    )

    return np.round(res[1], 5)

def k2(col, *, data=combine, group="site", group_val=["1", "2"]):
    res = stats.ks_2samp(
        data[data[group] == group_val[0]][col], 
        data[data[group] == group_val[1]][col]
    )

    return np.round(res[1], 5)


#predicted target values in train and test have different distributions
#Kolmogorov-Smirnov test and T test are used

for i in predicted_val:
    print(f'model, {i}')
    print('--ks test p value --')
    print(k2(i, data = combine, group = 'is_train', group_val = [0, 1]))
    print('--t test p-value --')
    print(ttest(i, data = combine, group = 'is_train', group_val = [0, 1]))
    print()

## Optimal ROC cutoff


Find optimal ROC cutoff for all models, total 14.

In [ ]:
def classify(y_true, y_pred, threshold):
    
    new_pred = y_pred >= threshold
    
    pos_pred = new_pred 
    neg_pred = 1-new_pred
    
    tp = np.sum(y_true * pos_pred)     #True Positive
    fp = np.sum((1-y_true) * pos_pred) #False Positive
    tn = np.sum((1-y_true) * neg_pred) #True Negative
    fn = np.sum(y_true * neg_pred)     #False Negative
    
    sensitivity = tp / (tp + fn)       #TP Ratio
    specificity = tn / (tn + fp)       #TN Ratio
    
    tpr = sensitivity
    fpr = fp/(fp + tn) #1 - specificity
    tnr = specificity
    
    return tpr, fpr, tnr, tp, tn

#simple ex
y = np.array([0, 0, 1, 1])
scores = np.array([0.1, 0.4, 0.35, 0.8])

tpr, fpr, tnr, tp, tn = classify(y, scores, 0.35)

tpr, fpr, tnr, tp, tn



In [ ]:
def opt_cutoff(train, feat, n_thresholds, do_print = True):
    #searching cutoff from min to max of predicted target values
    cutoff = np.linspace(train[feat].min(),
                         train[feat].max(),
                         n_thresholds)
    
    
    tpr = np.zeros(n_thresholds)
    fpr = np.zeros(n_thresholds)
    tnr = np.zeros(n_thresholds)
    
    for c, i in enumerate(cutoff):
        tpr1, fpr1, tnr1, tp, tn = classify(train['target'].values, train[feat].values, i)
        
        tpr[c] = tpr1
        fpr[c] = fpr1
        tnr[c] = tnr1
    
    optimal_idx = np.argmax(tpr + (-fpr))
    optimal_cutoff = cutoff[optimal_idx]
    
    
    new_tpr, new_fpr, new_tnr, tp, tn = classify(train['target'].values, train[feat].values, optimal_cutoff)
    
    if do_print:
        print(f'optimal cutoff, {optimal_cutoff}')
    
        plt.plot(fpr, tpr)
        plt.scatter(new_fpr, new_tpr)
        plt.title(f'{feat}, roc curve')
    
        print(f'tpr with cutoff {np.round(new_tpr, 5)}')
        print(f'fpr with cutoff {np.round(new_fpr, 5)}')
        print(f'tnr with cutoff {np.round(new_tnr, 5)}')
    
    feat_mean = train[feat].mean()
    feat_std = train[feat].std()
    
    return optimal_cutoff, feat_mean, feat_std

In [ ]:
optimal_cutoff, col_avg, col_std = opt_cutoff(train, predicted_val[1], 5000) 

In [ ]:
%%time
optimal_cutoff = np.zeros(len(predicted_val))
col_avg = np.zeros(len(predicted_val))
col_std = np.zeros(len(predicted_val))

for c, i in enumerate(tqdm(predicted_val)):
    oc, ca, cs = opt_cutoff(train, i, 10000, do_print = False)
    optimal_cutoff[c] = oc
    col_avg[c] = ca
    col_std[c] = cs

In [ ]:
print('========================')
print('Real target value counts')
print(train['target'].value_counts())
print('========================')


tpr_s = []
fpr_s = []
tnr_s = []
sc = []

for c, i in enumerate(predicted_val):
    
    tpr, fpr, tnr, tp, tn = classify(train['target'].values, train[i].values, optimal_cutoff[c])
    score = np.round(roc_auc_score(train['target'].values, train[i].values),4)
    
    tpr_s.append(tpr)
    fpr_s.append(fpr)
    tnr_s.append(tnr)
    sc.append(score)
    
    print()
    print('---------------------')
    print(f'Model :: {i}')
    print(f'True Negative       : {tn}')
    print(f'True Negative Ratio : {np.int(np.round(tnr,2) * 100)}%')
    print()
    print(f'True Positive       : {tp}')
    print(f'True Positive Ratio : {np.int(np.round(tpr,2) * 100)}%')
    print(f'False Positive Ratio: {np.int(np.round(fpr,2) * 100)}%')
    print()
    print(f'OOF AUC score           : {score}')
    print('---------------------')

In [ ]:
plt.figure(num=None, figsize=(10, 8), dpi=80)
plt.scatter(optimal_cutoff, col_avg)
for p, o, c in zip(predicted_val, optimal_cutoff, col_avg):
    plt.annotate(p, (o, c))
plt.title('Optimal Cutoff vs Predicted Value Average')

cor = np.round(np.corrcoef(optimal_cutoff, col_avg)[1,0], 7)


print(f'correlation between cutoff and prediction average, {cor}')
print('Linearity between those two seems acceptable, but not sure for now')

In [ ]:
plt.figure(num=None, figsize=(10, 8), dpi=80)
plt.scatter(optimal_cutoff, col_avg + col_std)
for p, o, ca, cs in zip(predicted_val, optimal_cutoff, col_avg, col_std):
    plt.annotate(p, (o, ca + cs))
plt.title('Optimal Cutoff vs (Predicted Value Average + Standard Deviation)')

cor = np.round(np.corrcoef(optimal_cutoff, col_avg + col_std)[1,0], 7)
print(f'correlation between cutoff and prediction average + prediction std, {cor}')
print('No patterns')

In [ ]:
plt.figure(num=None, figsize=(10, 8), dpi=80)
plt.scatter(optimal_cutoff-col_std, col_avg - col_std)
for p, o, ca, cs in zip(predicted_val, optimal_cutoff, col_avg, col_std):
    plt.annotate(p, (o - cs, ca - cs))

plt.title('(Optimal Cutoff - Std) vs (Predicted Value Average - Std)')

cor = np.round((np.corrcoef(optimal_cutoff-col_std, col_avg - col_std)[1,0]), 7)

print(f'correlation between cutoff - prediction std and prediction average - prediction std, {cor}')
print('Interesting pattern found here')

In [ ]:
te_col_avg = np.zeros(len(predicted_val))
te_col_std = np.zeros(len(predicted_val))

for c, i in enumerate(tqdm(predicted_val)):
    te_col_avg[c] = test[i].mean()
    te_col_std[c] = test[i].std()

tr = pd.DataFrame({'cutoff' : optimal_cutoff, 'avg': col_avg, 'std' : col_std, 'cutoff-avg' : optimal_cutoff - col_avg, 
                   'cutoff-std': optimal_cutoff - col_std, 'avg+std':col_avg + col_std, 'avg-std':col_avg - col_std})
te = pd.DataFrame({'avg': te_col_avg, 'std' : te_col_std, 'avg-std' : te_col_avg - te_col_std})

In [ ]:
plt.figure(num=None, figsize=(10, 8), dpi=80)
mask = np.zeros_like(tr.corr())
mask[np.triu_indices_from(mask)] = True
sns.heatmap(tr.corr(),mask = mask, annot = True)

In [ ]:
def mae(x,y):
    return np.sum(np.abs(x - y))/len(x)

print(mae(tr['cutoff-std'], tr['avg-std']))

## Predict ROC cutoff

Since there is high correlation between the ROC cutoff subtracted by standard deviation of predicted values and the average of predicted values subtracted by standard deviation of predicted values, we predict ROC cutoff subtracted by standard deviation of predicted values from the average of predicted values subtracted by standard deviation of predicted values.

In summary, 

Predict ROC cutoff - Predicted Std 

from Predicted Avg - Predicted Std.

In [ ]:
from xgboost import XGBRegressor

tree = XGBRegressor().fit(tr[['avg-std']], tr[['cutoff-std']])

score = np.int(np.round((tree.score(tr[['avg-std']], tr[['cutoff-std']]))*100, 0))

print(f'R2 score, {score}% variation explained')

tr_cutoff_std = tree.predict(tr[['avg-std']]).flatten()
te_cutoff_std = tree.predict(te[['avg-std']]).flatten()

#Since we predict the value of (ROC cutoff - target std) value,
#we add (target std) to the predicted (ROC cutoff - target std)
tr_cutoff = tr_cutoff_std + tr['std']
te_cutoff = te_cutoff_std + te['std']

tr_error = np.round(mae(tr_cutoff, optimal_cutoff), 8)
trte_error = np.round(mae(tr_cutoff, te_cutoff), 8)
trte_avg = np.round(mae(tr['avg'], te['avg']),8)

print(f'MAE,  predicted train optimal ROC cutoff vs true train optimal ROC cutoff     {tr_error}')
print(f'MAE,  predicted train optimal ROC cutoff vs predicted test optimal ROC cutoff {trte_error}')
print(f'MAE,  predicted target average for train vs predicted target average for test {trte_avg}')
print()
print('Gap of predicted ROC cutoff between train and test is simliar with the gap of predicted target average between train and test')

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(8,4), dpi=100)

sns.distplot(tr['avg'], hist = False, ax = ax1, label = 'train')
sns.distplot(te['avg'], hist = False, ax = ax1, label = 'test')
ax1.title.set_text('Average of Predicted target')

sns.distplot(tr_cutoff, hist = False, ax = ax2, label = 'train')
sns.distplot(te_cutoff, hist = False, ax = ax2, label = 'test')
ax2.title.set_text('ROC Cutoff of Predicted target')

Determine confident predicted test observation by predicted ROC cutoff for all models

In [ ]:
pseudo_test = pd.DataFrame([])
pseudo_test['image_name'] = test['image_name']

for c, i in enumerate(predicted_val):
    pseudo_test[i + '_target'] = (test[i] >= te_cutoff[c]) * 1
    
pseudo_test['sum_by_cutoff'] = pseudo_test.drop(['image_name'], axis = 1).apply(lambda x: x.sum(), axis = 1)
pseudo_test[['image_name', 'sum_by_cutoff']].sort_values(['sum_by_cutoff'])

In [ ]:
n_pos_pred = len(pseudo_test.loc[pseudo_test['sum_by_cutoff'] == 14, 'image_name'])

print(f'{n_pos_pred} confident predicted melanoma observations in test by all 14 models with predicted ROC cutoff')

In [ ]:
n_neg_pred = len(pseudo_test.loc[pseudo_test['sum_by_cutoff'] == 0, 'image_name'])

print(f'{n_neg_pred} confident predicted benign observations in test by all 14 models with predicted ROC cutoff')

## Conclusion


This found can be just for the predicted values from my models, or number of samples can be too small to ensure whether this works or not. So you should check with your predicted values if you want to give it a try this. 